In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import geopandas as gpd

import plotly
import folium
from folium import Choropleth
from folium.plugins import TimeSliderChoropleth

In [7]:
df = pd.read_csv("data/machine_learning_data_Final.csv")
df = df.drop(columns="Unnamed: 0")
df = df.rename(columns={"district":"UHFCODE"})

#Read the geoJSON file using geopandas
geojson = gpd.read_file("data/georef-united-states-of-america-county.geojson")

# The datasets use UHF42 shapefiles. We import those

uhf42shapes = gpd.read_file(
    "data/UHF_42_DOHMH_2009/UHF_42_DOHMH_2009.shp",
    dtypes = {"UHFCODE":"int"}
)
uhf42shapes["UHFCODE"] = uhf42shapes.UHFCODE.astype("int")
uhf42shapes.head()

# they are all here. We remove unecesary columns
geojson_ny = uhf42shapes[["UHFCODE", "geometry"]]
geojson_ny = geojson_ny[geojson_ny["UHFCODE"] != 0]
# geojson_ny = geojson_ny.rename(columns={"UHFCODE":"district"})
del geojson
del uhf42shapes


In [8]:
df

,year,UHFCODE,NO2,PM2_5,O3,SO2,asthma_hosp,Boroughs
0,2009,101,23.20,11.03,23.67,6.62,61,Bronx
1,2009,102,22.39,10.68,26.82,5.38,282,Bronx
2,2009,103,24.82,11.10,24.47,9.48,693,Bronx
3,2009,104,22.83,10.59,26.72,5.15,588,Bronx
4,2009,105,28.07,11.76,23.08,9.36,631,Bronx
...,...,...,...,...,...,...,...,...
331,2016,410,11.72,5.98,38.18,0.20,73,Queens
332,2016,501,17.47,7.30,33.42,0.12,83,Staten Island
333,2016,502,14.93,6.80,34.18,0.13,72,Staten Island
334,2016,503,14.11,7.10,34.68,0.12,12,Staten Island


In [14]:
years=[2009,2010,2011,2012,2013,2014,2015,2016]

In [15]:
lat, lon = 40.7, -73.95
map_ny = folium.Map(width=1000,height=1000,
    location=[lat, lon], zoom_start=11, tiles = None ,overlay=False,
    zoom_control=True,
    dragging=True,
    scrollWheelZoom=False,

)

m_2009 = folium.FeatureGroup(name='2009',overlay=False,show=True).add_to(map_ny)
m_2010 = folium.FeatureGroup(name='2010',overlay=False,show=False).add_to(map_ny)
m_2011 = folium.FeatureGroup(name='2011',overlay=False,show=False).add_to(map_ny)
m_2012 = folium.FeatureGroup(name='2012',overlay=False,show=False).add_to(map_ny)
m_2013 = folium.FeatureGroup(name='2013',overlay=False,show=False).add_to(map_ny)
m_2014 = folium.FeatureGroup(name='2014',overlay=False,show=False).add_to(map_ny)
m_2015 = folium.FeatureGroup(name='2015',overlay=False,show=False).add_to(map_ny)
m_2016 = folium.FeatureGroup(name='2016',overlay=False,show=False).add_to(map_ny)
group_dict = {2009:m_2009,
              2010:m_2010,
              2011:m_2011,
              2012:m_2012,
              2013:m_2013,
              2014:m_2014,
              2015:m_2015,
              2016:m_2016}

for year in years:
    group = group_dict[year]
    df_temp = df[df.year == year]
    df_temp = geojson_ny.merge(df_temp,on="UHFCODE")

    c = Choropleth(
    geo_data=df_temp,
    name='Choropleth',
    data=df_temp,
    columns=['UHFCODE','asthma_hosp'],
    key_on="properties.UHFCODE",
    fill_color='YlOrRd',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Asthma Hospitalizations',
    smooth_factor=0,
    highlight=True,
    overlay=True,
    show=False,
    ).geojson.add_to(group)

    c.add_child(
    folium.features.GeoJsonTooltip(['asthma_hosp',"PM2_5","NO2","O3","SO2"],aliases=["Asthma Hospitalizations","PM 2.5","NO2","O3","SO2"])
    )

folium.TileLayer('cartodbdark_matter',overlay=True,name="View in Dark Mode").add_to(map_ny)
folium.TileLayer('cartodbpositron',overlay=True,name="Viw in Light Mode").add_to(map_ny)
folium.LayerControl(collapsed=False).add_to(map_ny)
map_ny.save("map.html")
map_ny